In [18]:
%reset

In [5]:
import numpy as np
import h5py

In [20]:
# Path to your file
file_path = "ldpccode.txt"

# List to store extracted data
data = []

with open(file_path, "r") as file:
    for line in file:
        # Strip newline and split by whitespace
        values = line.strip().split()
        # Convert each value to integer
        values = [int(v) for v in values]
        # Add the list of values to the main list
        data.append(values)

# Example: printing the first few lines of extracted data
for i in range(10):  # Print first 10 lines
    print(data[i])


[22422, 10282, 11626, 19997, 11161, 2922, 3122, 99, 5625, 17064, 8270, 179]
[25087, 16218, 17015, 828, 20041, 25656, 4186, 11629, 22599, 17305, 22515, 6463]
[11049, 22853, 25706, 14388, 5500, 19245, 8732, 2177, 13555, 11346, 17265, 3069]
[16581, 22225, 12563, 19717, 23577, 11555, 25496, 6853, 25403, 5218, 15925, 21766]
[16529, 14487, 7643, 10715, 17442, 11119, 5679, 14155, 24213, 21000, 1116, 15620]
[5340, 8636, 16693, 1434, 5635, 6516, 9482, 20189, 1066, 15013, 25361, 14243]
[18506, 22236, 20912, 8952, 5421, 15691, 6126, 21595, 500, 6904, 13059, 6802]
[8433, 4694, 5524, 14216, 3685, 19721, 25420, 9937, 23813, 9047, 25651, 16826]
[21500, 24814, 6344, 17382, 7064, 13929, 4004, 16552, 12818, 8720, 5286, 2206]
[22517, 2429, 19065, 2921, 21611, 1873, 7507, 5661, 23006, 23128, 20543, 19777]


In [21]:
# LDPC code parameters
n_ldpc = 64800             # example for normal frame
rate = 3/5
k_ldpc = int(n_ldpc*rate)
N_p = n_ldpc - k_ldpc      # number of parity bits
q = 72                     # depends on code rate


In [22]:
info_bits = np.random.randint(0, 2, k_ldpc)

In [23]:
p = np.zeros(N_p, dtype=np.uint8)  # parity bits initialized to 0
H_matrix = np.zeros((N_p,n_ldpc), dtype=np.uint8)
# Example base parity addresses for i0 (from DVB table, e.g. rate 3/5)
for index in range(len(data)):
    base_addresses = np.array(data[index])

    # Process first bit i0
    i0 = info_bits[360*index+0]
    # print(360*q+0)
    for addr in base_addresses:
        p[addr] ^= i0
        H_matrix[addr,360*index+0] = 1

    # Process next bits i1 ... i359
    for m in range(1, 360):
        im = info_bits[360*index+m]
        # print(360*q+m)
        for addr in base_addresses:
            new_addr = (addr + (m % 360) * q) % N_p
            p[new_addr] ^= im
            H_matrix[new_addr,360*index+m] = 1

# Final codeword

for index in range(1,N_p):
    p[index] ^= p[index-1]


# Create an n x n zero matrix
P_matrix = np.zeros((N_p, N_p), dtype=int)

# Main diagonal = 1
np.fill_diagonal(P_matrix, 1)

# Subdiagonal (below main diagonal) = 1
for i in range(1, N_p):
    P_matrix[i, i-1] = 1  # (row, column)

H_matrix[:,k_ldpc:] = P_matrix

codeword = np.concatenate([info_bits,p])
# codeword = np.concatenate([p, info_bits])

In [24]:
len(data)

108

In [25]:
with h5py.File('check_mat_3_5.mat', 'r') as f:
    # HDF5 groups mirror MATLAB variables
    cw  = np.transpose(np.array(f['/CW']))
    # print(np.squeeze(np.array(f['/K'])))
    # k   = int(np.squeeze(np.array(f['/K'])))
    # n   = int(np.squeeze(np.array(f['/N'])))
    parity_matrix   = np.transpose(np.array(f['/p']))
    uw  = np.transpose(np.array(f['/u']))




In [ ]:
p_alternative = np.matmul(info_bits, np.transpose(H_matrix[:,:k_ldpc]))
for index in range(1,N_p):
    p_alternative[index] ^= p_alternative[index-1]

codeword_alt = np.concatenate([info_bits,p_alternative])

In [26]:
from scipy.sparse import csr_matrix

In [27]:
# Convert H to sparse format (only once, not inside a loop)
H_sparse = csr_matrix(H_matrix[:, :k_ldpc])

In [28]:
# Compute p_alternative using sparse matrix-vector multiplication
p_alternative = H_sparse.dot(info_bits) % 2  # mod 2 because XOR over GF(2)

# Cumulative XOR
p_alternative = np.mod(np.cumsum(p_alternative) , 2)

# Final codeword
codeword_alt = np.concatenate([info_bits, p_alternative])


In [8]:
print(cw.shape, cw.dtype, parity_matrix.shape, uw.shape)

(1000, 64800) uint8 (25920, 64800) (1000, 38880)


In [12]:
np.sum(np.mod(np.matmul(H_matrix[:1000,:],codeword),2))

0

In [11]:
np.sum(np.mod(np.matmul(parity_matrix[:1000,:],codeword),2))

0

In [29]:
np.sum(np.mod(np.matmul(H_matrix[:1000,:],codeword_alt),2))

0

In [30]:
np.sum(np.mod(np.matmul(parity_matrix[:1000,:],codeword_alt),2))

0

## Short 16200 LDPC codes

In [1]:
from scipy.sparse import csr_matrix
import numpy as np


In [2]:

# Example: printing the first few lines of extracted data
for i in range(10):  # Print first 10 lines
    print(data[i])


[0, 2084, 1613, 1548, 1286, 1460, 3196, 4297, 2481, 3369, 3451, 4620, 2622]
[1, 122, 1516, 3448, 2880, 1407, 1847, 3799, 3529, 373, 971, 4358, 3108]
[2, 259, 3399, 929, 2650, 864, 3996, 3833, 107, 5287, 164, 3125, 2350]
[3, 342, 3529]
[4, 4198, 2147]
[5, 1880, 4836]
[6, 3864, 4910]
[7, 243, 1542]
[8, 3011, 1436]
[9, 2167, 2512]


In [5]:
# LDPC code parameters
n_ldpc = 16200             # example for normal frame
rate = 2/3
q = 15                     # depends on code rate
file_path = "ldpccode_short_2_3.txt"
k_ldpc = int(n_ldpc * rate)

In [2]:
def generate_ldpc_H_matrix(n_ldpc: int, rate: float, q: int, file_path: str) -> np.ndarray:
    # Compute derived parameters
    k_ldpc = int(n_ldpc * rate)
    N_p = n_ldpc - k_ldpc  # number of parity bits

    data = []
    with open(file_path, "r") as file:
        for line in file:
            values = [int(v) for v in line.strip().split()]
            data.append(values)

    H_matrix = np.zeros((N_p, n_ldpc), dtype=np.uint8)

    for index, base_addresses in enumerate(data):
        base_addresses = np.array(base_addresses)

        for addr in base_addresses:
            H_matrix[addr, 360 * index + 0] = 1

        for m in range(1, 360):
            for addr in base_addresses:
                new_addr = (addr + (m % 360) * q) % N_p
                H_matrix[new_addr, 360 * index + m] = 1

    P_matrix = np.zeros((N_p, N_p), dtype=int)
    np.fill_diagonal(P_matrix, 1)
    for i in range(1, N_p):
        P_matrix[i, i - 1] = 1

    H_matrix[:, k_ldpc:] = P_matrix

    return H_matrix


In [3]:
H_matrix = generate_ldpc_H_matrix(
    n_ldpc=16200,
    rate=2/3,
    q=15,
    file_path="ldpccode_short_2_3.txt"
)

print(H_matrix.shape)


(5400, 16200)


In [3]:
k_ldpc = int(n_ldpc*rate)
N_p = n_ldpc - k_ldpc      # number of parity bits

# List to store extracted data
data = []

with open(file_path, "r") as file:
    for line in file:
        # Strip newline and split by whitespace
        values = line.strip().split()
        # Convert each value to integer
        values = [int(v) for v in values]
        # Add the list of values to the main list
        data.append(values)

H_matrix = np.zeros((N_p,n_ldpc), dtype=np.uint8)
# Example base parity addresses for i0 (from DVB table, e.g. rate 3/5)
for index in range(len(data)):
    base_addresses = np.array(data[index])
    for addr in base_addresses:
        H_matrix[addr,360*index+0] = 1

    for m in range(1, 360):
        for addr in base_addresses:
            new_addr = (addr + (m % 360) * q) % N_p
            H_matrix[new_addr,360*index+m] = 1

# Create an n x n zero matrix
P_matrix = np.zeros((N_p, N_p), dtype=int)
np.fill_diagonal(P_matrix, 1)
# Subdiagonal (below main diagonal) = 1
for i in range(1, N_p):
    P_matrix[i, i-1] = 1  # (row, column)

H_matrix[:,k_ldpc:] = P_matrix

In [6]:
info_bits = np.random.randint(0, 2, k_ldpc)
H_sparse = csr_matrix(H_matrix[:, :k_ldpc])

In [7]:
# Compute p_alternative using sparse matrix-vector multiplication
p_alternative = H_sparse.dot(info_bits) % 2  # mod 2 because XOR over GF(2)
# Cumulative XOR
p_alternative = np.mod(np.cumsum(p_alternative) , 2)
# Final codeword
codeword_alt = np.concatenate([info_bits, p_alternative])